# Identyfikacja oddziaływań neutrin w detektorach ciekłoargonowych metodami uczenia maszynowego

## 1. Przygotowanie danych

Użycie GENIE do generacji zdarzeń zgodnie z instrukcją zawartą w pracy w rozdziale 3.1 powinno skutkować utworzeniem pliku o nazwie gntp.0.ghep.root. Plik ten należy zamienić na format o nazwie RooTracker komendą:

Z tak przygotowanego pliku można wyodrębnić najpotrzebniejsze dane do pliku .csv

In [ ]:
import csv